In [117]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

def model_comparison(df, target_column, test_size=0.2, random_state=42, cv=5):
    # Prepare the data
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Dictionary of models and their reduced hyperparameter grids
    models = {
        'Logistic Regression': (LogisticRegression(max_iter=1000), {
            'C': [0.01, 0.1, 1],
            'solver': ['liblinear', 'lbfgs'],
            'class_weight': [None, 'balanced']
        }),
        'KNN': (KNeighborsClassifier(), {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance'],
            'p': [1, 2]
        }),
        'Decision Tree': (DecisionTreeClassifier(), {
            'max_depth': [None, 10, 20, 40],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }),
        'Random Forest': (RandomForestClassifier(), {
            'n_estimators': [100, 200, 400],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [2, 4]
        }),
        'Gradient Boosting': (GradientBoostingClassifier(), {
            'n_estimators': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4]
        }),
        'XGBoost': (XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'), {
            'n_estimators': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4],
            'gamma': [0, 0.1]
        }),
        'LightGBM': (LGBMClassifier(), {
            'n_estimators': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [31, 50, 70]
        }),
        'CatBoost': (CatBoostClassifier(verbose=0), {
            'iterations': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'depth': [4, 6, 8]
        })
    }
    # Dictionary to store results
    results = {}

    for name, (model, param_grid) in models.items():
        start_time = time.time()
        
        # Perform GridSearchCV
        grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train)
        
        # Get the best model
        best_model = grid_search.best_estimator_

        # Measure training time for best parameters
        best_param_train_start = time.time()
        best_model.fit(X_train_scaled, y_train)
        best_param_train_time = time.time() - best_param_train_start

        # Measure inference time for best parameters
        inference_start_time = time.time()
        y_pred = best_model.predict(X_test_scaled)
        inference_time = time.time() - inference_start_time
        
        # Calculate total computation time
        computation_time = time.time() - start_time
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        if len(np.unique(y)) == 2:  # Binary classification
            auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1])
        else:  # Multiclass classification
            auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr', average='macro')

        # Perform cross-validation
        cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=cv, scoring='accuracy')

        results[name] = {
            'Accuracy': accuracy,
            'AUC Score': auc,
            'CV Mean Accuracy': np.mean(cv_scores),
            'CV Std Accuracy': np.std(cv_scores),
            'Training Time (Best Params)': best_param_train_time,
            'Inference Time (Best Params)': inference_time,
            'Computation Time (Total)': computation_time,
            'Best Parameters': grid_search.best_params_
        }

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results).T

    return result_df


In [118]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.neural_network import MLPClassifier

def mlp_comparison(X, y, result_df, test_size=0.2, random_state=42, cv=3):
    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define the MLP model and its hyperparameter grid
    mlp = MLPClassifier(max_iter=1000, random_state=random_state)
    param_grid = {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive']
    }

    # Perform GridSearchCV with StratifiedKFold
    start_time = time.time()
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)
    grid_search = GridSearchCV(mlp, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Measure training time for best parameters
    best_param_train_start = time.time()
    best_model.fit(X_train_scaled, y_train)
    training_time = time.time() - best_param_train_start

    # Measure inference time for best parameters
    inference_start_time = time.time()
    y_pred = best_model.predict(X_test_scaled)
    inference_time = time.time() - inference_start_time

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    if len(np.unique(y)) == 2:  # Binary classification
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1])
    else:  # Multiclass classification
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr', average='macro')

    # Perform cross-validation
    cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=cv, scoring='accuracy')

    # Store results in the existing result DataFrame
    result_df.loc['MLP'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': np.mean(cv_scores),
        'CV Std Accuracy': np.std(cv_scores),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': grid_search.best_params_
    }

    return result_df, grid_search.best_params_

In [119]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def dnn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    # Define the DNN model
    class DNN(nn.Module):
        def __init__(self, input_dim, hidden_dims, output_dim):
            super(DNN, self).__init__()
            layers = []
            for hidden_dim in hidden_dims:
                layers.append(nn.Linear(input_dim, hidden_dim))
                layers.append(nn.ReLU())
                input_dim = hidden_dim
            layers.append(nn.Linear(input_dim, output_dim))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)

    # Define the objective function for Optuna
    def objective(trial):
        # Define hyperparameters to tune
        hidden_dims = [trial.suggest_int(f'hidden_dim_{i}', 32, 256) for i in range(3)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the model
        input_dim = X_train.shape[1]
        output_dim = len(np.unique(y))
        model = DNN(input_dim, hidden_dims, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Start timing the entire process
    start_time = time.time()

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final model with the best hyperparameters
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    best_model = DNN(input_dim, [best_params[f'hidden_dim_{i}'] for i in range(3)], output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        # Convert to numpy for metric calculation
        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:  # Multi-class classification
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = DNN(input_dim, [best_params[f'hidden_dim_{i}'] for i in range(3)], output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['DNN'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params

In [120]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def dcn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class CrossLayer(nn.Module):
        def __init__(self, input_dim):
            super(CrossLayer, self).__init__()
            self.weight = nn.Parameter(torch.Tensor(input_dim, 1))
            self.bias = nn.Parameter(torch.Tensor(input_dim, 1))
            nn.init.xavier_uniform_(self.weight)
            nn.init.zeros_(self.bias)

        def forward(self, x0, x):
            x = x.unsqueeze(2)
            x0 = x0.unsqueeze(2)
            interaction = torch.matmul(x0, torch.matmul(x.transpose(1, 2), self.weight))
            return x0.squeeze(2) + interaction.squeeze(2) + self.bias.T

    class DCN(nn.Module):
        def __init__(self, input_dim, cross_layers, hidden_layers, output_dim):
            super(DCN, self).__init__()
            self.cross_layers = nn.ModuleList([CrossLayer(input_dim) for _ in range(cross_layers)])
            
            deep_layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                deep_layers.append(nn.ReLU())
            self.deep_net = nn.Sequential(*deep_layers)
            
            self.final_layer = nn.Linear(input_dim + hidden_layers[-1], output_dim)

        def forward(self, x):
            cross_out = x
            for layer in self.cross_layers:
                cross_out = layer(x, cross_out)
            deep_out = self.deep_net(x)
            concat_out = torch.cat([cross_out, deep_out], dim=1)
            return self.final_layer(concat_out)

    def objective(trial):
        cross_layers = trial.suggest_int('cross_layers', 1, 5)
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        input_dim = X_train.shape[1]
        output_dim = len(np.unique(y))
        model = DCN(input_dim, cross_layers, hidden_layers, output_dim).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    start_time = time.time()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params

    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    best_model = DCN(input_dim, best_params['cross_layers'], 
                     [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                     output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = DCN(input_dim, best_params['cross_layers'], 
                         [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                         output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    computation_time = time.time() - start_time

    result_df.loc['DCN'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params



In [121]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def wide_and_deep_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class WideAndDeepNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers, output_dim):
            super(WideAndDeepNetwork, self).__init__()
            
            # Wide part
            self.wide = nn.Linear(input_dim, output_dim)
            
            # Deep part
            deep_layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                deep_layers.append(nn.ReLU())
            deep_layers.append(nn.Linear(hidden_layers[-1], output_dim))
            self.deep = nn.Sequential(*deep_layers)

        def forward(self, x):
            wide_out = self.wide(x)
            deep_out = self.deep(x)
            return wide_out + deep_out

    def objective(trial):
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        input_dim = X_train.shape[1]
        output_dim = len(np.unique(y))
        model = WideAndDeepNetwork(input_dim, hidden_layers, output_dim).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    start_time = time.time()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params

    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    best_model = WideAndDeepNetwork(input_dim, 
                                    [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                                    output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = WideAndDeepNetwork(input_dim, 
                                        [best_params[f'hidden_layer_{i}'] for i in range(3)], 
                                        output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    computation_time = time.time() - start_time

    result_df.loc['Wide_and_Deep'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params

In [122]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier
import optuna

def xgb_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    start_time = time.time()

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers, output_dim):
            super(NeuralNetwork, self).__init__()
            layers = []
            prev_dim = input_dim
            for hidden_dim in hidden_layers:
                layers.append(nn.Linear(prev_dim, hidden_dim))
                layers.append(nn.ReLU())
                prev_dim = hidden_dim
            layers.append(nn.Linear(prev_dim, output_dim))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)

    def objective(trial):
        # Define hyperparameters to tune for XGBoost
        xgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
        }

        # Train XGBoost model
        xgb_model = XGBClassifier(**xgb_params, use_label_encoder=False, eval_metric='mlogloss')
        xgb_model.fit(X_train_scaled, y_train)
        
        # Extract features using XGBoost
        X_train_transformed = xgb_model.apply(X_train_scaled)
        X_test_transformed = xgb_model.apply(X_test_scaled)
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        y_train_tensor = torch.LongTensor(y_train).to(device)
        y_test_tensor = torch.LongTensor(y_test).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = []
        for i in range(3):  # Allow up to 3 hidden layers
            if trial.suggest_categorical(f'use_hidden_layer_{i}', [True, False]):
                hidden_layers.append(trial.suggest_int(f'hidden_layer_{i}', 32, 256))
        
        nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        output_dim = len(np.unique(y))
        model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_transformed_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final XGBoost model with the best hyperparameters
    xgb_best_params = {
        'n_estimators': best_params['n_estimators'],
        'max_depth': best_params['max_depth'],
        'learning_rate': best_params['xgb_learning_rate'],
        'subsample': best_params['subsample'],
        'colsample_bytree': best_params['colsample_bytree']
    }
    xgb_model = XGBClassifier(**xgb_best_params, use_label_encoder=False, eval_metric='mlogloss')
    xgb_model.fit(X_train_scaled, y_train)

    # Extract features using XGBoost
    X_train_transformed = xgb_model.apply(X_train_scaled)
    X_test_transformed = xgb_model.apply(X_test_scaled)

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }

    for i in range(3):  # Assuming max 3 hidden layers
        if f'use_hidden_layer_{i}' in best_params and best_params[f'use_hidden_layer_{i}']:
            nn_best_params['hidden_layers'].append(best_params[f'hidden_layer_{i}'])

    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:  # Multi-class classification
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        xgb_model.fit(X_fold_train, y_fold_train)
        X_fold_train_transformed = xgb_model.apply(X_fold_train)
        X_fold_val_transformed = xgb_model.apply(X_fold_val)

        X_fold_train_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    computation_time = time.time() - start_time

    result_df.loc['XGBoost + NN'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [123]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from lightgbm import LGBMClassifier
import optuna

def lgbm_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers, output_dim):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], output_dim))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)

    def objective(trial):
        # Define hyperparameters to tune for LightGBM
        lgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('lgb_learning_rate', 1e-4, 1e-1, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
        }

        # Train LightGBM model
        lgb_model = LGBMClassifier(**lgb_params)
        lgb_model.fit(X_train_scaled, y_train)
        
        # Extract features using LightGBM
        X_train_transformed = lgb_model.predict_proba(X_train_scaled)
        X_test_transformed = lgb_model.predict_proba(X_test_scaled)
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        y_train_tensor = torch.LongTensor(y_train).to(device)
        y_test_tensor = torch.LongTensor(y_test).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        nn_learning_rate = trial.suggest_float('nn_learning_rate', 1e-4, 1e-1, log=True)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        output_dim = len(np.unique(y))
        model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_transformed_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final LightGBM model with the best hyperparameters
    lgb_best_params = {
        'n_estimators': best_params['n_estimators'],
        'max_depth': best_params['max_depth'],
        'learning_rate': best_params['lgb_learning_rate'],
        'num_leaves': best_params['num_leaves'],
        'subsample': best_params['subsample'],
        'colsample_bytree': best_params['colsample_bytree']
    }
    lgb_model = LGBMClassifier(**lgb_best_params)
    lgb_model.fit(X_train_scaled, y_train)

    # Extract features using LightGBM
    X_train_transformed = lgb_model.predict_proba(X_train_scaled)
    X_test_transformed = lgb_model.predict_proba(X_test_scaled)

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:  # Multi-class classification
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        lgb_model.fit(X_fold_train, y_fold_train)
        X_fold_train_transformed = lgb_model.predict_proba(X_fold_train)
        X_fold_val_transformed = lgb_model.predict_proba(X_fold_val)

        X_fold_train_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['LightGBM + NN'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params

In [124]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def autoint_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class AutoInt(nn.Module):
        def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
            super(AutoInt, self).__init__()
            self.embedding = nn.Linear(input_dim, embedding_dim)
            self.attention_layers = nn.ModuleList([
                nn.MultiheadAttention(embedding_dim, num_heads) for _ in range(num_layers)
            ])
            self.fc = nn.Linear(embedding_dim, 1)

        def forward(self, x):
            x = self.embedding(x).unsqueeze(1)
            for attn_layer in self.attention_layers:
                x, _ = attn_layer(x, x, x)
            x = x.squeeze(1)
            x = self.fc(x)
            return x

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers, output_dim):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], output_dim))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)

    def objective(trial):
        # Define hyperparameters to tune for AutoInt
        num_heads = trial.suggest_int('num_heads', 1, 8)
        embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        
        # Train AutoInt model
        autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        
        for epoch in range(10):  # Fixed number of epochs for AutoInt
            autoint_model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = autoint_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
        
        # Extract features using AutoInt
        autoint_model.eval()
        with torch.no_grad():
            X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
            X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        output_dim = len(np.unique(y))
        model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_transformed_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final AutoInt model with the best hyperparameters
    embedding_dim = best_params['embedding_dim']
    num_heads = best_params['num_heads']
    num_layers = best_params['num_layers']
    autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor.float().unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for epoch in range(10):  # Fixed number of epochs for AutoInt
        autoint_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = autoint_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Extract features using AutoInt
    autoint_model.eval()
    with torch.no_grad():
        X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:  # Multi-class classification
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_autoint_model = AutoInt(X_fold_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        fold_optimizer = optim.Adam(fold_autoint_model.parameters(), lr=0.001)
        fold_criterion = nn.MSELoss()
        fold_train_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor.float().unsqueeze(1))
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=32, shuffle=True)

        for epoch in range(10):  # Fixed number of epochs for AutoInt
            fold_autoint_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_autoint_model(batch_X)
                loss = fold_criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_autoint_model.eval()
        with torch.no_grad():
            X_fold_train_transformed = fold_autoint_model.embedding(X_fold_train_tensor).cpu().numpy()
            X_fold_val_transformed = fold_autoint_model.embedding(X_fold_val_tensor).cpu().numpy()

        X_fold_train_transformed_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        X_fold_val_transformed_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_train_dataset = TensorDataset(X_fold_train_transformed_tensor, y_fold_train_tensor)
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_transformed_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['AutoInt + NN'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params

In [125]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def ft_transformer_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class FTTransformer(nn.Module):
        def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
            super(FTTransformer, self).__init__()
            self.embedding = nn.Linear(input_dim, embedding_dim)
            self.transformer_layers = nn.ModuleList([
                nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads) for _ in range(num_layers)
            ])
            self.fc = nn.Linear(embedding_dim, input_dim)

        def forward(self, x):
            x = self.embedding(x).unsqueeze(1)
            for transformer_layer in self.transformer_layers:
                x = transformer_layer(x)
            x = x.squeeze(1)
            x = self.fc(x)
            return x

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers, output_dim):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], output_dim))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)

    def objective(trial):
        # Define hyperparameters to tune for FT-Transformer
        num_heads = trial.suggest_int('num_heads', 1, 8)
        embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        
        # Train FT-Transformer model
        ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        
        for epoch in range(10):  # Fixed number of epochs for FT-Transformer
            ft_transformer_model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = ft_transformer_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
        
        # Extract features using FT-Transformer
        ft_transformer_model.eval()
        with torch.no_grad():
            X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
            X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        output_dim = len(np.unique(y))
        model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_transformed_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final FT-Transformer model with the best hyperparameters
    embedding_dim = best_params['embedding_dim']
    num_heads = best_params['num_heads']
    num_layers = best_params['num_layers']
    ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for epoch in range(10):  # Fixed number of epochs for FT-Transformer
        ft_transformer_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = ft_transformer_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Extract features using FT-Transformer
    ft_transformer_model.eval()
    with torch.no_grad():
        X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }
    input_dim = X_train_transformed.shape[1]
    output_dim = len(np.unique(y))
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_transformed_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:  # Multi-class classification
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_ft_transformer_model = FTTransformer(X_fold_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        fold_optimizer = optim.Adam(fold_ft_transformer_model.parameters(), lr=0.001)
        fold_criterion = nn.CrossEntropyLoss()

        fold_train_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=32, shuffle=True)

        for epoch in range(10):
            fold_ft_transformer_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_ft_transformer_model(batch_X)
                loss = fold_criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_ft_transformer_model.eval()
        with torch.no_grad():
            X_fold_train_transformed = fold_ft_transformer_model.embedding(X_fold_train_tensor).cpu().numpy()
            X_fold_val_transformed = fold_ft_transformer_model.embedding(X_fold_val_tensor).cpu().numpy()

        X_fold_train_transformed_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        X_fold_val_transformed_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers'], output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_train_dataset = TensorDataset(X_fold_train_transformed_tensor, y_fold_train_tensor)
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_transformed_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['FT-Transformer'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [126]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def neural_architecture_search(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers, output_dim):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], output_dim))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)

    def objective(trial):
        # Define hyperparameters to tune for Neural Network
        num_layers = trial.suggest_int('num_layers', 1, 5)
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(num_layers)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train.shape[1]
        output_dim = len(np.unique(y))
        model = NeuralNetwork(input_dim, hidden_layers, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final Neural Network model with the best hyperparameters
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    best_model = NeuralNetwork(input_dim, 
                               [best_params[f'hidden_layer_{i}'] for i in range(best_params['num_layers'])], 
                               output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        inference_time = time.time() - inference_start_time

        y_pred = predicted.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            proba = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            auc = roc_auc_score(y_true, proba)
        else:  # Multi-class classification
            proba = torch.softmax(outputs, dim=1).cpu().numpy()
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = NeuralNetwork(input_dim, 
                                   [best_params[f'hidden_layer_{i}'] for i in range(best_params['num_layers'])], 
                                   output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['Neural Architecture Search'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [142]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def node_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class NODE(nn.Module):
        def __init__(self, input_dim, num_layers, num_trees, tree_dim, output_dim):
            super(NODE, self).__init__()
            self.layers = nn.ModuleList()
            for _ in range(num_layers):
                layer = nn.ModuleList()
                for _ in range(num_trees):
                    tree = nn.Sequential(
                        nn.Linear(input_dim, tree_dim),
                        nn.ReLU(),
                        nn.Linear(tree_dim, 1)
                    )
                    layer.append(tree)
                self.layers.append(layer)
            self.output = nn.Linear(num_layers * num_trees, output_dim)

        def forward(self, x):
            tree_outputs = []
            for layer in self.layers:
                layer_outputs = []
                for tree in layer:
                    layer_outputs.append(tree(x))
                layer_output = torch.cat(layer_outputs, dim=1)
                tree_outputs.append(layer_output)
            x = torch.cat(tree_outputs, dim=1)
            return self.output(x)

    def objective(trial):
        # Define hyperparameters to tune for NODE
        num_layers = trial.suggest_int('num_layers', 1, 5)
        num_trees = trial.suggest_int('num_trees', 1, 10)
        tree_dim = trial.suggest_int('tree_dim', 8, 64)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the NODE model
        input_dim = X_train.shape[1]
        output_dim = len(np.unique(y))
        model = NODE(input_dim, num_layers, num_trees, tree_dim, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final NODE model with the best hyperparameters
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    best_model = NODE(input_dim, 
                      best_params['num_layers'], 
                      best_params['num_trees'], 
                      best_params['tree_dim'], 
                      output_dim).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        # Convert to numpy for metric calculation
        y_true = y_test_tensor.cpu().numpy()
        
        _, predicted = torch.max(outputs, 1)
        y_pred = predicted.cpu().numpy()
        proba = torch.softmax(outputs, dim=1).cpu().numpy()

        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            auc = roc_auc_score(y_true, proba[:, 1])
        else:  # Multi-class classification
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = NODE(input_dim, 
                          best_params['num_layers'], 
                          best_params['num_trees'], 
                          best_params['tree_dim'], 
                          output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['NODE'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [143]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna

def tabnet_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    def objective(trial):
        # Define hyperparameters to tune for TabNet
        n_d = trial.suggest_int('n_d', 8, 64)
        n_a = trial.suggest_int('n_a', 8, 64)
        n_steps = trial.suggest_int('n_steps', 3, 10)
        gamma = trial.suggest_float('gamma', 1.0, 2.0)
        lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the TabNet model
        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=learning_rate),
            device_name=device
        )

        # Training
        model.fit(
            X_train=X_train_scaled, y_train=y_train,
            eval_set=[(X_test_scaled, y_test)],
            eval_name=['val'],
            eval_metric=['accuracy'],
            max_epochs=num_epochs,
            patience=10,
            batch_size=batch_size,
            virtual_batch_size=batch_size // 2,
            num_workers=0,
            drop_last=False
        )

        # Evaluation
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final TabNet model with the best hyperparameters
    best_model = TabNetClassifier(
        n_d=best_params['n_d'],
        n_a=best_params['n_a'],
        n_steps=best_params['n_steps'],
        gamma=best_params['gamma'],
        lambda_sparse=best_params['lambda_sparse'],
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=best_params['learning_rate']),
        device_name=device
    )

    training_start_time = time.time()
    best_model.fit(
        X_train=X_train_scaled, y_train=y_train,
        eval_set=[(X_test_scaled, y_test)],
        eval_name=['val'],
        eval_metric=['accuracy'],
        max_epochs=best_params['num_epochs'],
        patience=10,
        batch_size=best_params['batch_size'],
        virtual_batch_size=best_params['batch_size'] // 2,
        num_workers=0,
        drop_last=False
    )
    training_time = time.time() - training_start_time

    # Evaluation
    y_pred = best_model.predict(X_test_scaled)
    inference_start_time = time.time()
    y_pred_proba = best_model.predict_proba(X_test_scaled)
    inference_time = time.time() - inference_start_time

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    if len(np.unique(y)) == 2:  # Binary classification
        auc = roc_auc_score(y_test, y_pred_proba[:, 1])
    else:  # Multiclass classification
        auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        fold_model = TabNetClassifier(
            n_d=best_params['n_d'],
            n_a=best_params['n_a'],
            n_steps=best_params['n_steps'],
            gamma=best_params['gamma'],
            lambda_sparse=best_params['lambda_sparse'],
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=best_params['learning_rate']),
            device_name=device
        )

        fold_model.fit(
            X_train=X_fold_train, y_train=y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            eval_name=['val'],
            eval_metric=['accuracy'],
            max_epochs=best_params['num_epochs'],
            patience=10,
            batch_size=best_params['batch_size'],
            virtual_batch_size=best_params['batch_size'] // 2,
            num_workers=0,
            drop_last=False
        )

        y_fold_pred = fold_model.predict(X_fold_val)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_pred)
        cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['TabNet'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [144]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def kan_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class KAN(nn.Module):
        def __init__(self, input_dim, hidden_dim, output_dim):
            super(KAN, self).__init__()
            self.hidden_layer = nn.Linear(input_dim, hidden_dim)
            self.activation = nn.ReLU()
            self.output_layer = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            h = self.activation(self.hidden_layer(x))
            out = self.output_layer(h)
            return out

    def objective(trial):
        # Define hyperparameters to tune for KAN
        hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the KAN model
        input_dim = X_train.shape[1]
        output_dim = len(np.unique(y))
        model = KAN(input_dim, hidden_dim, output_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final KAN model with the best hyperparameters
    input_dim = X_train.shape[1]
    output_dim = len(np.unique(y))
    best_model = KAN(input_dim, 
                     best_params['hidden_dim'], 
                     output_dim).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        # Convert to numpy for metric calculation
        y_true = y_test_tensor.cpu().numpy()
        
        _, predicted = torch.max(outputs, 1)
        y_pred = predicted.cpu().numpy()
        proba = torch.softmax(outputs, dim=1).cpu().numpy()

        accuracy = accuracy_score(y_true, y_pred)
        
        if output_dim == 2:  # Binary classification
            auc = roc_auc_score(y_true, proba[:, 1])
        else:  # Multi-class classification
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = KAN(input_dim, best_params['hidden_dim'], output_dim).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['KAN'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [145]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import optuna

def saint_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.LongTensor(y_test).to(device)

    class SAINT(nn.Module):
        def __init__(self, input_dim, num_classes, dim, depth, heads, mlp_dim, dropout=0.1):
            super(SAINT, self).__init__()
            self.embeds = nn.Linear(input_dim, dim)
            self.transformer = nn.TransformerEncoder(
                nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout),
                num_layers=depth
            )
            self.mlp_head = nn.Sequential(
                nn.LayerNorm(dim),
                nn.Linear(dim, num_classes)
            )

        def forward(self, x):
            x = self.embeds(x)
            x = x.unsqueeze(1)  # Add sequence dimension
            x = self.transformer(x)
            x = x.squeeze(1)  # Remove sequence dimension
            return self.mlp_head(x)

    def objective(trial):
        # Define hyperparameters to tune for SAINT
        heads = trial.suggest_int('heads', 1, 8)
        dim = trial.suggest_int('dim', heads, 256, step=heads)  # Ensure dim is divisible by heads
        depth = trial.suggest_int('depth', 1, 6)
        mlp_dim = trial.suggest_int('mlp_dim', 32, 256)
        dropout = trial.suggest_float('dropout', 0.0, 0.5)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the SAINT model
        input_dim = X_train.shape[1]
        num_classes = len(np.unique(y))
        model = SAINT(input_dim, num_classes, dim, depth, heads, mlp_dim, dropout).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predicted.cpu().numpy())

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final SAINT model with the best hyperparameters
    input_dim = X_train.shape[1]
    num_classes = len(np.unique(y))
    best_model = SAINT(input_dim, num_classes, 
                       best_params['dim'], 
                       best_params['depth'], 
                       best_params['heads'], 
                       best_params['mlp_dim'], 
                       best_params['dropout']).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        outputs = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        # Convert to numpy for metric calculation
        y_true = y_test_tensor.cpu().numpy()
        
        _, predicted = torch.max(outputs, 1)
        y_pred = predicted.cpu().numpy()
        proba = torch.softmax(outputs, dim=1).cpu().numpy()

        accuracy = accuracy_score(y_true, y_pred)
        
        if num_classes == 2:  # Binary classification
            auc = roc_auc_score(y_true, proba[:, 1])
        else:  # Multi-class classification
            auc = roc_auc_score(y_true, proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.LongTensor(y_fold_train).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.LongTensor(y_fold_val).to(device)

        fold_model = SAINT(input_dim, num_classes, 
                           best_params['dim'], 
                           best_params['depth'], 
                           best_params['heads'], 
                           best_params['mlp_dim'], 
                           best_params['dropout']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            outputs = fold_model(X_fold_val_tensor)
            _, predicted = torch.max(outputs, 1)
            fold_accuracy = accuracy_score(y_fold_val_tensor.cpu().numpy(), predicted.cpu().numpy())
            cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['SAINT'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params



In [146]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna

def vime_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    def objective(trial):
        # Define hyperparameters to tune for VIME-like model (using TabNet as proxy)
        n_d = trial.suggest_int('n_d', 8, 64)
        n_a = trial.suggest_int('n_a', 8, 64)
        n_steps = trial.suggest_int('n_steps', 3, 10)
        gamma = trial.suggest_float('gamma', 1.0, 2.0)
        lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the TabNet model
        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=learning_rate),
            device_name=device
        )

        # Training
        model.fit(
            X_train=X_train_scaled, y_train=y_train,
            eval_set=[(X_test_scaled, y_test)],
            eval_name=['val'],
            eval_metric=['accuracy'],
            max_epochs=num_epochs,
            patience=10,
            batch_size=batch_size,
            virtual_batch_size=batch_size // 2,
            num_workers=0,
            drop_last=False
        )

        # Evaluation
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        return accuracy

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final TabNet model with the best hyperparameters
    best_model = TabNetClassifier(
        n_d=best_params['n_d'],
        n_a=best_params['n_a'],
        n_steps=best_params['n_steps'],
        gamma=best_params['gamma'],
        lambda_sparse=best_params['lambda_sparse'],
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=best_params['learning_rate']),
        device_name=device
    )

    training_start_time = time.time()
    best_model.fit(
        X_train=X_train_scaled, y_train=y_train,
        eval_set=[(X_test_scaled, y_test)],
        eval_name=['val'],
        eval_metric=['accuracy'],
        max_epochs=best_params['num_epochs'],
        patience=10,
        batch_size=best_params['batch_size'],
        virtual_batch_size=best_params['batch_size'] // 2,
        num_workers=0,
        drop_last=False
    )
    training_time = time.time() - training_start_time

    # Evaluation
    y_pred = best_model.predict(X_test_scaled)
    inference_start_time = time.time()
    y_pred_proba = best_model.predict_proba(X_test_scaled)
    inference_time = time.time() - inference_start_time

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    if len(np.unique(y)) == 2:  # Binary classification
        auc = roc_auc_score(y_test, y_pred_proba[:, 1])
    else:  # Multiclass classification
        auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

    # Cross-validation
    cv_scores = []
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        fold_model = TabNetClassifier(
            n_d=best_params['n_d'],
            n_a=best_params['n_a'],
            n_steps=best_params['n_steps'],
            gamma=best_params['gamma'],
            lambda_sparse=best_params['lambda_sparse'],
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=best_params['learning_rate']),
            device_name=device
        )

        fold_model.fit(
            X_train=X_fold_train, y_train=y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            eval_name=['val'],
            eval_metric=['accuracy'],
            max_epochs=best_params['num_epochs'],
            patience=10,
            batch_size=best_params['batch_size'],
            virtual_batch_size=best_params['batch_size'] // 2,
            num_workers=0,
            drop_last=False
        )

        y_fold_pred = fold_model.predict(X_fold_val)
        fold_accuracy = accuracy_score(y_fold_val, y_fold_pred)
        cv_scores.append(fold_accuracy)

    cv_mean_accuracy = np.mean(cv_scores)
    cv_std_accuracy = np.std(cv_scores)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['VIME'] = {
        'Accuracy': accuracy,
        'AUC Score': auc,
        'CV Mean Accuracy': cv_mean_accuracy,
        'CV Std Accuracy': cv_std_accuracy,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [147]:
import numpy as np
import pandas as pd
from scipy.stats import yeojohnson

def yeojohnson_transform(X):
    if isinstance(X, pd.DataFrame):
        X_transformed = X.copy()
        for column in X_transformed.columns:
            X_transformed[column], _ = yeojohnson(X_transformed[column])
    else:
        X = np.asarray(X)
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        X_transformed = np.zeros_like(X)
        for i in range(X.shape[1]):
            X_transformed[:, i], _ = yeojohnson(X[:, i])
    
    return X_transformed

# Function call example
def apply_yeojohnson(X, y):
    X_transformed = yeojohnson_transform(X)
    return X_transformed, y

In [148]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

def apply_smote_to_training(X, y, test_size=0.2, random_state=42):

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    

    # Apply SMOTE to the training data only
    smote = SMOTE(random_state=random_state)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Combine the SMOTE-augmented training data with the original test data
    X_combined = np.vstack((X_train_smote, X_test))
    y_combined = np.concatenate((y_train_smote, y_test))
    
    # If X was originally a DataFrame, convert X_combined back to DataFrame
    if isinstance(X, pd.DataFrame):
        X_combined = pd.DataFrame(X_combined, columns=X.columns)
    
    return X_combined, y_combined




In [149]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def encode_categorical_data(df, target_column='Y'):
    df_encoded = df.copy()
    le = LabelEncoder()
    categorical_cols = df_encoded.select_dtypes(include=['object', 'category']).columns
    categorical_cols = [col for col in categorical_cols if col != target_column]
    
    for column in categorical_cols:
        if df_encoded[column].dtype == 'object' or df_encoded[column].dtype.name == 'category':
            df_encoded[column] = le.fit_transform(df_encoded[column].astype(str))
    
    df_encoded[target_column], _ = pd.factorize(df_encoded[target_column])
    
    return df_encoded

In [150]:
file_prefix = "apple"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
if len(df) >= 1405:
    df = df.sample(n=1405, random_state=42, replace=True)
df =  encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_yeojohnson(X, y)
X, y = apply_smote_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/classification/{file_prefix}_result.csv', index=True)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:28:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:28:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:28:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\

KeyboardInterrupt: 

In [ ]:
file_prefix = "heart"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
if len(df) >= 1405:
    df = df.sample(n=1405, random_state=42, replace=True)
df =  encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_yeojohnson(X, y)
X, y = apply_smote_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/classification/{file_prefix}_result.csv', index=True)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:34:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:34:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:34:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\

[LightGBM] [Info] Number of positive: 109, number of negative: 133
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000171 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 246
[LightGBM] [Info] Number of data points in the train set: 242, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.450413 -> initscore=-0.199001
[LightGBM] [Info] Start training from score -0.199001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2024-08-05 14:36:46,489] A new study created in memory with name: no-name-eced5878-3fb8-44cb-84aa-1c48d5eb66e7


                     Accuracy AUC Score CV Mean Accuracy CV Std Accuracy  \
Logistic Regression  0.836066  0.927802         0.826276         0.05041   
KNN                  0.836066  0.931573         0.838435        0.045479   
Decision Tree        0.852459  0.854526         0.760544        0.047249   
Random Forest        0.868852   0.93319         0.813861         0.03034   
Gradient Boosting    0.786885   0.90194         0.809694        0.028769   
XGBoost              0.770492   0.90625         0.814031        0.013309   
LightGBM             0.868852   0.91056         0.814031        0.013309   
CatBoost             0.836066  0.920259          0.83869        0.033851   
MLP                  0.815385  0.902462         0.861668        0.040573   

                    Training Time (Best Params) Inference Time (Best Params)  \
Logistic Regression                    0.001995                          0.0   
KNN                                    0.000998                     0.001994   

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 14:36:47,409] Trial 0 finished with value: 0.7846153846153846 and parameters: {'hidden_dim_0': 99, 'hidden_dim_1': 112, 'hidden_dim_2': 105, 'learning_rate': 0.018460524339003227, 'batch_size': 256, 'num_epochs': 64}. Best is trial 0 with value: 0.7846153846153846.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 14:36:48,549] Trial 1 

...

epoch 0  | loss: 4.14642 | val_accuracy: 0.55769 |  0:00:00s
epoch 1  | loss: 2.75714 | val_accuracy: 0.53846 |  0:00:00s
epoch 2  | loss: 1.12296 | val_accuracy: 0.59615 |  0:00:00s
epoch 3  | loss: 0.9625  | val_accuracy: 0.59615 |  0:00:00s
epoch 4  | loss: 0.95455 | val_accuracy: 0.75    |  0:00:00s
epoch 5  | loss: 0.62419 | val_accuracy: 0.71154 |  0:00:01s
epoch 6  | loss: 0.70716 | val_accuracy: 0.78846 |  0:00:01s
epoch 7  | loss: 0.49354 | val_accuracy: 0.71154 |  0:00:01s
epoch 8  | loss: 0.67833 | val_accuracy: 0.76923 |  0:00:01s
epoch 9  | loss: 0.6036  | val_accuracy: 0.80769 |  0:00:01s
epoch 10 | loss: 0.53694 | val_accuracy: 0.80769 |  0:00:02s
epoch 11 | loss: 0.4517  | val_accuracy: 0.86538 |  0:00:02s
epoch 12 | loss: 0.6546  | val_accuracy: 0.80769 |  0:00:02s
epoch 13 | loss: 0.45669 | val_accuracy: 0.80769 |  0:00:02s
epoch 14 | loss: 0.49719 | val_accuracy: 0.80769 |  0:00:02s
epoch 15 | loss: 0.53364 | val_accuracy: 0.82692 |  0:00:02s
epoch 16 | loss: 0.52556

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-08-05 14:43:06,023] A new study created in memory with name: no-name-9598d981-beaa-4f94-87a1-4fea65c63b62


epoch 21 | loss: 0.4276  | val_accuracy: 0.75    |  0:00:03s

Early stopping occurred at epoch 21 with best_epoch = 11 and best_val_accuracy = 0.86538
                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.836066  0.927802         0.826276   
KNN                         0.836066  0.931573         0.838435   
Decision Tree               0.852459  0.854526         0.760544   
Random Forest               0.868852   0.93319         0.813861   
Gradient Boosting           0.786885   0.90194         0.809694   
XGBoost                     0.770492   0.90625         0.814031   
LightGBM                    0.868852   0.91056         0.814031   
CatBoost                    0.836066  0.920259          0.83869   
MLP                         0.815385  0.902462         0.861668   
DNN                         0.830769  0.910985         0.807692   
DCN                              0.8  0.903409         0.807692   
Wide_and_Deep               0.738462  0.88541

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\3744686281.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-05 14:43:11,471] Trial 0 finished with value: 0.8 and parameters: {'heads': 1, 'dim': 171, 'depth': 3, 'mlp_dim': 124, 'dropout': 0.24620502729414845, 'learning_rate': 0.0028975788628894093, 'batch_size': 256, 'num_epochs': 61}. Best is trial 0 with value: 

                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.836066  0.927802         0.826276   
KNN                         0.836066  0.931573         0.838435   
Decision Tree               0.852459  0.854526         0.760544   
Random Forest               0.868852   0.93319         0.813861   
Gradient Boosting           0.786885   0.90194         0.809694   
XGBoost                     0.770492   0.90625         0.814031   
LightGBM                    0.868852   0.91056         0.814031   
CatBoost                    0.836066  0.920259          0.83869   
MLP                         0.815385  0.902462         0.861668   
DNN                         0.830769  0.910985         0.807692   
DCN                              0.8  0.903409         0.807692   
Wide_and_Deep               0.738462  0.885417         0.830769   
XGBoost + NN                0.846154  0.926136         0.811538   
LightGBM + NN               0.846154  0.912879         0.82692

In [137]:
file_prefix = "loan"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
if len(df) >= 1405:
    df = df.sample(n=1405, random_state=42, replace=True)
df =  encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_yeojohnson(X, y)
X, y = apply_smote_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/classification/{file_prefix}_result.csv', index=True)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:46:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:46:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:46:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\

[LightGBM] [Info] Number of positive: 416, number of negative: 708
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000418 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1311
[LightGBM] [Info] Number of data points in the train set: 1124, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.370107 -> initscore=-0.531759
[LightGBM] [Info] Start training from score -0.531759
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[I 2024-08-05 14:52:23,220] A new study created in memory with name: no-name-30aa07d0-1988-4bb5-8a07-0e7927680111


                     Accuracy AUC Score CV Mean Accuracy CV Std Accuracy  \
Logistic Regression  0.925267  0.977813         0.933286        0.015355   
KNN                  0.918149  0.976972         0.902119        0.011749   
Decision Tree        0.960854  0.955485         0.968857        0.007968   
Random Forest        0.982206  0.998431         0.975968        0.006707   
Gradient Boosting    0.967972  0.994453         0.975968        0.010775   
XGBoost              0.967972  0.998123         0.978639        0.010316   
LightGBM             0.992883  0.999496         0.979544        0.008233   
CatBoost             0.982206  0.998879         0.976865        0.007123   
MLP                  0.979532  0.996847         0.961966        0.006262   

                    Training Time (Best Params) Inference Time (Best Params)  \
Logistic Regression                    0.005984                          0.0   
KNN                                    0.005984                     0.012966   

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 14:52:26,409] Trial 0 finished with value: 0.9590643274853801 and parameters: {'hidden_dim_0': 59, 'hidden_dim_1': 223, 'hidden_dim_2': 46, 'learning_rate': 0.0013419667841887818, 'batch_size': 128, 'num_epochs': 48}. Best is trial 0 with value: 0.9590643274853801.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 14:52:30,465] Trial 1 

...

epoch 0  | loss: 0.75833 | val_accuracy: 0.71795 |  0:00:00s
epoch 1  | loss: 0.34478 | val_accuracy: 0.87912 |  0:00:01s
epoch 2  | loss: 0.24007 | val_accuracy: 0.91941 |  0:00:01s
epoch 3  | loss: 0.17123 | val_accuracy: 0.92674 |  0:00:02s
epoch 4  | loss: 0.19451 | val_accuracy: 0.94505 |  0:00:03s
epoch 5  | loss: 0.16308 | val_accuracy: 0.91941 |  0:00:03s
epoch 6  | loss: 0.17113 | val_accuracy: 0.91209 |  0:00:04s
epoch 7  | loss: 0.18351 | val_accuracy: 0.91209 |  0:00:04s
epoch 8  | loss: 0.14936 | val_accuracy: 0.92674 |  0:00:05s
epoch 9  | loss: 0.15847 | val_accuracy: 0.91575 |  0:00:05s
epoch 10 | loss: 0.18791 | val_accuracy: 0.92308 |  0:00:06s
epoch 11 | loss: 0.16746 | val_accuracy: 0.92674 |  0:00:06s
epoch 12 | loss: 0.14353 | val_accuracy: 0.91209 |  0:00:07s
epoch 13 | loss: 0.13819 | val_accuracy: 0.91209 |  0:00:08s
epoch 14 | loss: 0.14064 | val_accuracy: 0.92308 |  0:00:08s

Early stopping occurred at epoch 14 with best_epoch = 4 and best_val_accuracy = 0.94

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-08-05 15:39:34,802] A new study created in memory with name: no-name-21d6a05e-e73c-4481-822b-edd45365ceca


                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.925267  0.977813         0.933286   
KNN                         0.918149  0.976972         0.902119   
Decision Tree               0.960854  0.955485         0.968857   
Random Forest               0.982206  0.998431         0.975968   
Gradient Boosting           0.967972  0.994453         0.975968   
XGBoost                     0.967972  0.998123         0.978639   
LightGBM                    0.992883  0.999496         0.979544   
CatBoost                    0.982206  0.998879         0.976865   
MLP                         0.979532  0.996847         0.961966   
DNN                         0.976608  0.994105         0.956097   
DCN                         0.973684  0.994893         0.952442   
Wide_and_Deep               0.976608  0.997772         0.966335   
XGBoost + NN                0.982456  0.998766          0.96853   
LightGBM + NN               0.982456  0.993214         0.97511

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\3744686281.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-05 15:43:45,628] Trial 0 finished with value: 0.8391812865497076 and parameters: {'heads': 4, 'dim': 212, 'depth': 5, 'mlp_dim': 185, 'dropout': 0.2060749310863922, 'learning_rate': 0.0007646500984566662, 'batch_size': 32, 'num_epochs': 92}. Best is trial 0

                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.925267  0.977813         0.933286   
KNN                         0.918149  0.976972         0.902119   
Decision Tree               0.960854  0.955485         0.968857   
Random Forest               0.982206  0.998431         0.975968   
Gradient Boosting           0.967972  0.994453         0.975968   
XGBoost                     0.967972  0.998123         0.978639   
LightGBM                    0.992883  0.999496         0.979544   
CatBoost                    0.982206  0.998879         0.976865   
MLP                         0.979532  0.996847         0.961966   
DNN                         0.976608  0.994105         0.956097   
DCN                         0.973684  0.994893         0.952442   
Wide_and_Deep               0.976608  0.997772         0.966335   
XGBoost + NN                0.982456  0.998766          0.96853   
LightGBM + NN               0.982456  0.993214         0.97511

In [138]:
file_prefix = "machine"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
if len(df) >= 1405:
    df = df.sample(n=1405, random_state=42, replace=True)
df =  encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_yeojohnson(X, y)
X, y = apply_smote_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/classification/{file_prefix}_result.csv', index=True)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:51:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:51:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:51:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\

[LightGBM] [Info] Number of positive: 449, number of negative: 306
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000115 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 216
[LightGBM] [Info] Number of data points in the train set: 755, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.594702 -> initscore=0.383438
[LightGBM] [Info] Start training from score 0.383438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[I 2024-08-05 15:55:16,136] A new study created in memory with name: no-name-2d335f0b-bcf2-489d-ac4a-c66b9fcd442a


                     Accuracy AUC Score CV Mean Accuracy CV Std Accuracy  \
Logistic Regression  0.867725  0.945684         0.915232        0.019376   
KNN                  0.857143  0.911652         0.917881        0.014868   
Decision Tree        0.846561  0.893002         0.886093        0.019823   
Random Forest        0.883598  0.954586         0.915232        0.016962   
Gradient Boosting     0.89418  0.955601         0.913907        0.011082   
XGBoost              0.888889  0.956277         0.913907        0.016754   
LightGBM             0.862434  0.935542         0.911258        0.027337   
CatBoost             0.878307  0.949966         0.919205         0.02347   
MLP                  0.883721  0.960239         0.921736        0.005868   

                    Training Time (Best Params) Inference Time (Best Params)  \
Logistic Regression                    0.004987                     0.000997   
KNN                                    0.000997                     0.002992   

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 15:55:19,176] Trial 0 finished with value: 0.9023255813953488 and parameters: {'hidden_dim_0': 141, 'hidden_dim_1': 86, 'hidden_dim_2': 223, 'learning_rate': 0.030142039671637005, 'batch_size': 64, 'num_epochs': 44}. Best is trial 0 with value: 0.9023255813953488.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 15:55:22,393] Trial 1 f

...

epoch 0  | loss: 2.32747 | val_accuracy: 0.51462 |  0:00:00s
epoch 1  | loss: 2.03111 | val_accuracy: 0.51462 |  0:00:01s
epoch 2  | loss: 1.74432 | val_accuracy: 0.56725 |  0:00:01s
epoch 3  | loss: 1.50115 | val_accuracy: 0.5848  |  0:00:02s
epoch 4  | loss: 1.17137 | val_accuracy: 0.62573 |  0:00:02s
epoch 5  | loss: 1.10357 | val_accuracy: 0.63743 |  0:00:03s
epoch 6  | loss: 0.95991 | val_accuracy: 0.67251 |  0:00:04s
epoch 7  | loss: 0.82014 | val_accuracy: 0.69591 |  0:00:04s
epoch 8  | loss: 0.74791 | val_accuracy: 0.73684 |  0:00:05s
epoch 9  | loss: 0.61677 | val_accuracy: 0.75439 |  0:00:05s
epoch 10 | loss: 0.57564 | val_accuracy: 0.78947 |  0:00:06s
epoch 11 | loss: 0.56688 | val_accuracy: 0.77778 |  0:00:06s
epoch 12 | loss: 0.53995 | val_accuracy: 0.80117 |  0:00:07s
epoch 13 | loss: 0.47725 | val_accuracy: 0.82456 |  0:00:08s
epoch 14 | loss: 0.48994 | val_accuracy: 0.82456 |  0:00:08s
epoch 15 | loss: 0.48189 | val_accuracy: 0.85965 |  0:00:09s
epoch 16 | loss: 0.44553

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-08-05 16:18:14,155] A new study created in memory with name: no-name-bd0fa84b-a905-4d94-ad41-9547195cdef0


                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.867725  0.945684         0.915232   
KNN                         0.857143  0.911652         0.917881   
Decision Tree               0.846561  0.893002         0.886093   
Random Forest               0.883598  0.954586         0.915232   
Gradient Boosting            0.89418  0.955601         0.913907   
XGBoost                     0.888889  0.956277         0.913907   
LightGBM                    0.862434  0.935542         0.911258   
CatBoost                    0.878307  0.949966         0.919205   
MLP                         0.883721  0.960239         0.921736   
DNN                         0.883721  0.935248         0.908867   
DCN                         0.893023  0.963574         0.912362   
Wide_and_Deep               0.883721  0.960152         0.913518   
XGBoost + NN                0.893023  0.940662         0.907738   
LightGBM + NN               0.897674  0.961452         0.91471

c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [3, 256] and step=3, but the range is not divisible by `step`. It will be replaced by [3, 255].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\3744686281.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-05 16:19:34,699] Trial 0 f

                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.867725  0.945684         0.915232   
KNN                         0.857143  0.911652         0.917881   
Decision Tree               0.846561  0.893002         0.886093   
Random Forest               0.883598  0.954586         0.915232   
Gradient Boosting            0.89418  0.955601         0.913907   
XGBoost                     0.888889  0.956277         0.913907   
LightGBM                    0.862434  0.935542         0.911258   
CatBoost                    0.878307  0.949966         0.919205   
MLP                         0.883721  0.960239         0.921736   
DNN                         0.883721  0.935248         0.908867   
DCN                         0.893023  0.963574         0.912362   
Wide_and_Deep               0.883721  0.960152         0.913518   
XGBoost + NN                0.893023  0.940662         0.907738   
LightGBM + NN               0.897674  0.961452         0.91471

In [139]:
file_prefix = "red"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
if len(df) >= 1405:
    df = df.sample(n=1405, random_state=42, replace=True)
df =  encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_yeojohnson(X, y)
X, y = apply_smote_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/classification/{file_prefix}_result.csv', index=True)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:28:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:28:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:28:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\

[LightGBM] [Info] Number of positive: 537, number of negative: 587
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000167 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 918
[LightGBM] [Info] Number of data points in the train set: 1124, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.477758 -> initscore=-0.089027
[LightGBM] [Info] Start training from score -0.089027
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[I 2024-08-05 16:35:40,073] A new study created in memory with name: no-name-7e66aa0c-fa00-4ed2-8535-11f2c2ee2d83


                     Accuracy AUC Score CV Mean Accuracy CV Std Accuracy  \
Logistic Regression  0.736655  0.815283         0.754448        0.007097   
KNN                  0.886121  0.959857         0.869222        0.016732   
Decision Tree        0.871886  0.873841         0.847861        0.020758   
Random Forest        0.882562  0.958151         0.881683        0.021651   
Gradient Boosting     0.88968  0.950891         0.885246        0.015404   
XGBoost              0.896797  0.952674         0.878127        0.012646   
LightGBM             0.886121  0.952929         0.875452        0.012179   
CatBoost             0.907473  0.957412         0.885238        0.020116   
MLP                  0.880546  0.903029         0.856153        0.010632   

                    Training Time (Best Params) Inference Time (Best Params)  \
Logistic Regression                    0.003988                          0.0   
KNN                                    0.002992                     0.006981   

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 16:35:45,648] Trial 0 finished with value: 0.8703071672354948 and parameters: {'hidden_dim_0': 198, 'hidden_dim_1': 150, 'hidden_dim_2': 140, 'learning_rate': 0.009509590924985739, 'batch_size': 128, 'num_epochs': 76}. Best is trial 0 with value: 0.8703071672354948.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 16:35:47,101] Trial 1

...

epoch 0  | loss: 3.57096 | val_accuracy: 0.49785 |  0:00:01s
epoch 1  | loss: 2.57699 | val_accuracy: 0.59227 |  0:00:03s
epoch 2  | loss: 1.99501 | val_accuracy: 0.6309  |  0:00:05s
epoch 3  | loss: 1.52672 | val_accuracy: 0.67811 |  0:00:06s
epoch 4  | loss: 1.21139 | val_accuracy: 0.72532 |  0:00:08s
epoch 5  | loss: 1.08047 | val_accuracy: 0.70386 |  0:00:10s
epoch 6  | loss: 1.02252 | val_accuracy: 0.73391 |  0:00:11s
epoch 7  | loss: 0.92226 | val_accuracy: 0.69528 |  0:00:13s
epoch 8  | loss: 0.86331 | val_accuracy: 0.71245 |  0:00:14s
epoch 9  | loss: 0.90707 | val_accuracy: 0.74249 |  0:00:16s
epoch 10 | loss: 0.87984 | val_accuracy: 0.72103 |  0:00:18s
epoch 11 | loss: 0.76381 | val_accuracy: 0.6824  |  0:00:20s
epoch 12 | loss: 0.91947 | val_accuracy: 0.72103 |  0:00:21s
epoch 13 | loss: 0.75675 | val_accuracy: 0.73391 |  0:00:23s
epoch 14 | loss: 0.7701  | val_accuracy: 0.74249 |  0:00:24s
epoch 15 | loss: 0.7361  | val_accuracy: 0.7382  |  0:00:26s
epoch 16 | loss: 0.75436

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-08-05 17:10:54,327] A new study created in memory with name: no-name-825c5f65-4825-4a79-a241-076c30222408


                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.736655  0.815283         0.754448   
KNN                         0.886121  0.959857         0.869222   
Decision Tree               0.871886  0.873841         0.847861   
Random Forest               0.882562  0.958151         0.881683   
Gradient Boosting            0.88968  0.950891         0.885246   
XGBoost                     0.896797  0.952674         0.878127   
LightGBM                    0.886121  0.952929         0.875452   
CatBoost                    0.907473  0.957412         0.885238   
MLP                         0.880546  0.903029         0.856153   
DNN                         0.887372  0.925489          0.87415   
DCN                          0.87372  0.921202         0.871571   
Wide_and_Deep                0.87372  0.924697         0.875841   
XGBoost + NN                 0.87372  0.931687         0.858758   
LightGBM + NN                0.87372  0.901911         0.88700

c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 256] and step=5, but the range is not divisible by `step`. It will be replaced by [5, 255].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\3744686281.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-05 17:11:06,568] Trial 0 f

                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.736655  0.815283         0.754448   
KNN                         0.886121  0.959857         0.869222   
Decision Tree               0.871886  0.873841         0.847861   
Random Forest               0.882562  0.958151         0.881683   
Gradient Boosting            0.88968  0.950891         0.885246   
XGBoost                     0.896797  0.952674         0.878127   
LightGBM                    0.886121  0.952929         0.875452   
CatBoost                    0.907473  0.957412         0.885238   
MLP                         0.880546  0.903029         0.856153   
DNN                         0.887372  0.925489          0.87415   
DCN                          0.87372  0.921202         0.871571   
Wide_and_Deep                0.87372  0.924697         0.875841   
XGBoost + NN                 0.87372  0.931687         0.858758   
LightGBM + NN                0.87372  0.901911         0.88700

In [140]:
file_prefix = "water"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
if len(df) >= 1405:
    df = df.sample(n=1405, random_state=42, replace=True)
df =  encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_yeojohnson(X, y)
X, y = apply_smote_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/classification/{file_prefix}_result.csv', index=True)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:31:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:31:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:31:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\

[LightGBM] [Info] Number of positive: 218, number of negative: 906
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000305 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2770
[LightGBM] [Info] Number of data points in the train set: 1124, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.193950 -> initscore=-1.424544
[LightGBM] [Info] Start training from score -1.424544
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[I 2024-08-05 17:41:09,387] A new study created in memory with name: no-name-c6e22a97-ab89-41a9-a261-943d4039e325


                     Accuracy AUC Score CV Mean Accuracy CV Std Accuracy  \
Logistic Regression  0.871886  0.861541         0.862099        0.008892   
KNN                  0.864769  0.887051         0.854075        0.010557   
Decision Tree        0.928826  0.900774         0.944849        0.009936   
Random Forest        0.939502  0.983462          0.92794        0.012979   
Gradient Boosting     0.94306  0.991027         0.956413        0.009873   
XGBoost              0.953737  0.992435         0.962647        0.016988   
LightGBM             0.953737  0.991291         0.963536        0.014147   
CatBoost              0.97153  0.994194         0.953754        0.013908   
MLP                  0.985748  0.998273         0.956548         0.00803   

                    Training Time (Best Params) Inference Time (Best Params)  \
Logistic Regression                    0.002993                          0.0   
KNN                                    0.001996                     0.004986   

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 17:41:13,648] Trial 0 finished with value: 0.9786223277909739 and parameters: {'hidden_dim_0': 134, 'hidden_dim_1': 221, 'hidden_dim_2': 164, 'learning_rate': 0.06916851521910337, 'batch_size': 128, 'num_epochs': 41}. Best is trial 0 with value: 0.9786223277909739.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 17:41:30,353] Trial 1 

...

epoch 0  | loss: 1.38353 | val_accuracy: 0.68452 |  0:00:01s
epoch 1  | loss: 0.54795 | val_accuracy: 0.77083 |  0:00:03s
epoch 2  | loss: 0.5152  | val_accuracy: 0.77381 |  0:00:04s
epoch 3  | loss: 0.47752 | val_accuracy: 0.75298 |  0:00:06s
epoch 4  | loss: 0.39811 | val_accuracy: 0.79464 |  0:00:08s
epoch 5  | loss: 0.39928 | val_accuracy: 0.80655 |  0:00:09s
epoch 6  | loss: 0.37572 | val_accuracy: 0.8244  |  0:00:11s
epoch 7  | loss: 0.34347 | val_accuracy: 0.84821 |  0:00:13s
epoch 8  | loss: 0.31888 | val_accuracy: 0.83631 |  0:00:14s
epoch 9  | loss: 0.3636  | val_accuracy: 0.8006  |  0:00:16s
epoch 10 | loss: 0.3283  | val_accuracy: 0.86607 |  0:00:18s
epoch 11 | loss: 0.34345 | val_accuracy: 0.81548 |  0:00:19s
epoch 12 | loss: 0.3532  | val_accuracy: 0.8244  |  0:00:21s
epoch 13 | loss: 0.35629 | val_accuracy: 0.77083 |  0:00:22s
epoch 14 | loss: 0.39899 | val_accuracy: 0.80357 |  0:00:24s
epoch 15 | loss: 0.38545 | val_accuracy: 0.83036 |  0:00:26s
epoch 16 | loss: 0.38895

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-08-05 18:19:30,051] A new study created in memory with name: no-name-e7a172ef-cfc8-4756-8e34-740280264f1b


                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.871886  0.861541         0.862099   
KNN                         0.864769  0.887051         0.854075   
Decision Tree               0.928826  0.900774         0.944849   
Random Forest               0.939502  0.983462          0.92794   
Gradient Boosting            0.94306  0.991027         0.956413   
XGBoost                     0.953737  0.992435         0.962647   
LightGBM                    0.953737  0.991291         0.963536   
CatBoost                     0.97153  0.994194         0.953754   
MLP                         0.985748  0.998273         0.956548   
DNN                         0.988124  0.997841          0.96131   
DCN                         0.978622  0.997909         0.963095   
Wide_and_Deep               0.978622  0.997205         0.961905   
XGBoost + NN                0.541568       0.5         0.704762   
LightGBM + NN               0.978622  0.992512           0.962

c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [7, 256] and step=7, but the range is not divisible by `step`. It will be replaced by [7, 252].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\3744686281.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-05 18:19:36,117] Trial 0 f

                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.871886  0.861541         0.862099   
KNN                         0.864769  0.887051         0.854075   
Decision Tree               0.928826  0.900774         0.944849   
Random Forest               0.939502  0.983462          0.92794   
Gradient Boosting            0.94306  0.991027         0.956413   
XGBoost                     0.953737  0.992435         0.962647   
LightGBM                    0.953737  0.991291         0.963536   
CatBoost                     0.97153  0.994194         0.953754   
MLP                         0.985748  0.998273         0.956548   
DNN                         0.988124  0.997841          0.96131   
DCN                         0.978622  0.997909         0.963095   
Wide_and_Deep               0.978622  0.997205         0.961905   
XGBoost + NN                0.541568       0.5         0.704762   
LightGBM + NN               0.978622  0.992512           0.962

In [141]:
file_prefix = "weather"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
df =  encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_yeojohnson(X, y)
X, y = apply_smote_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/classification/{file_prefix}_result.csv', index=True)

c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:37:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:37:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:37:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 639
[LightGBM] [Info] Number of data points in the train set: 1124, number of used features: 10
[LightGBM] [Info] Start training from score -1.281646
[LightGBM] [Info] Start training from score -1.426227
[LightGBM] [Info] Start training from score -1.415177
[LightGBM] [Info] Start training from score -1.429938
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

[I 2024-08-05 18:47:51,147] A new study created in memory with name: no-name-55628b14-fa4c-4317-91b5-25ed556d7757


                     Accuracy AUC Score CV Mean Accuracy CV Std Accuracy  \
Logistic Regression  0.846975  0.936392         0.847877        0.012241   
KNN                  0.914591  0.976787         0.908385        0.018048   
Decision Tree        0.925267  0.951957         0.910147        0.018065   
Random Forest        0.946619  0.995741         0.926171        0.015496   
Gradient Boosting    0.935943  0.994383         0.928849        0.017494   
XGBoost              0.935943  0.994944         0.931516         0.01599   
LightGBM             0.939502  0.996228         0.930623        0.015492   
CatBoost             0.932384  0.993246         0.922611        0.017416   
MLP                   0.92691  0.989907             0.93        0.012416   

                    Training Time (Best Params) Inference Time (Best Params)  \
Logistic Regression                    0.010972                     0.000996   
KNN                                    0.002992                     0.005983   

C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 18:47:52,365] Trial 0 finished with value: 0.9335548172757475 and parameters: {'hidden_dim_0': 155, 'hidden_dim_1': 81, 'hidden_dim_2': 190, 'learning_rate': 0.005087570447831714, 'batch_size': 128, 'num_epochs': 17}. Best is trial 0 with value: 0.9335548172757475.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\681114551.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-08-05 18:47:55,950] Trial 1 

...

epoch 0  | loss: 1.68508 | val_accuracy: 0.62917 |  0:00:00s
epoch 1  | loss: 0.93315 | val_accuracy: 0.75833 |  0:00:01s
epoch 2  | loss: 0.71862 | val_accuracy: 0.80833 |  0:00:01s
epoch 3  | loss: 0.64224 | val_accuracy: 0.825   |  0:00:02s
epoch 4  | loss: 0.58843 | val_accuracy: 0.825   |  0:00:03s
epoch 5  | loss: 0.54892 | val_accuracy: 0.8625  |  0:00:04s
epoch 6  | loss: 0.51558 | val_accuracy: 0.87917 |  0:00:04s
epoch 7  | loss: 0.4844  | val_accuracy: 0.88333 |  0:00:05s
epoch 8  | loss: 0.47551 | val_accuracy: 0.85833 |  0:00:06s
epoch 9  | loss: 0.46249 | val_accuracy: 0.85833 |  0:00:06s
epoch 10 | loss: 0.44723 | val_accuracy: 0.8625  |  0:00:07s
epoch 11 | loss: 0.47626 | val_accuracy: 0.86667 |  0:00:07s
epoch 12 | loss: 0.40574 | val_accuracy: 0.85    |  0:00:08s
epoch 13 | loss: 0.43343 | val_accuracy: 0.8625  |  0:00:09s
epoch 14 | loss: 0.43196 | val_accuracy: 0.87917 |  0:00:09s
epoch 15 | loss: 0.41089 | val_accuracy: 0.87083 |  0:00:10s
epoch 16 | loss: 0.36974

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-08-05 19:15:56,150] A new study created in memory with name: no-name-a90e6a8a-012b-4352-9501-16f2c24a6b75


                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.846975  0.936392         0.847877   
KNN                         0.914591  0.976787         0.908385   
Decision Tree               0.925267  0.951957         0.910147   
Random Forest               0.946619  0.995741         0.926171   
Gradient Boosting           0.935943  0.994383         0.928849   
XGBoost                     0.935943  0.994944         0.931516   
LightGBM                    0.939502  0.996228         0.930623   
CatBoost                    0.932384  0.993246         0.922611   
MLP                          0.92691  0.989907             0.93   
DNN                         0.933555  0.979618         0.925833   
DCN                         0.930233  0.979752           0.9225   
Wide_and_Deep                0.92691  0.980429           0.9225   
XGBoost + NN                 0.92691  0.973848         0.931667   
LightGBM + NN               0.953488  0.982328           0.937

c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [3, 256] and step=3, but the range is not divisible by `step`. It will be replaced by [3, 255].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_732\3744686281.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-05 19:16:00,466] Trial 0 f

                            Accuracy AUC Score CV Mean Accuracy  \
Logistic Regression         0.846975  0.936392         0.847877   
KNN                         0.914591  0.976787         0.908385   
Decision Tree               0.925267  0.951957         0.910147   
Random Forest               0.946619  0.995741         0.926171   
Gradient Boosting           0.935943  0.994383         0.928849   
XGBoost                     0.935943  0.994944         0.931516   
LightGBM                    0.939502  0.996228         0.930623   
CatBoost                    0.932384  0.993246         0.922611   
MLP                          0.92691  0.989907             0.93   
DNN                         0.933555  0.979618         0.925833   
DCN                         0.930233  0.979752           0.9225   
Wide_and_Deep                0.92691  0.980429           0.9225   
XGBoost + NN                 0.92691  0.973848         0.931667   
LightGBM + NN               0.953488  0.982328           0.937